### Skalowanie cech

Skalowanie wartości cechy wykonuje się nastepująco:
Obliczenie wartości minimalnej $X_{min}$ i maksymalnej $X_{max}$ cechy i przeskalowanie wartości do zakresu [0;1]
$$X_{[0;1]} = \frac{X-X_{min}}{X_{max}-X{min}}$$

Nastepnie przeskalowanie otrzymanych wartości z zakresu [0;1] do żądanego nowego zakresu $[X_{s,min};X_{s,max}]$:

$$X_s = X_{[0;1]}\cdot(X_{s,max}-X_{s,min}) + X_{s,min}$$

Skalowanie cech polega na przeliczeniu wartości cechy na wartości z danego zakresu, zwykle od 0 do 1 (wówczas $X_s=X_{[0;1]}$). Operację taką wykonuje się, gdyż w przypadku części modeli ML, większe wartości oznaczają bardziej istotny wpływ cechy na wynik predykcji. Jest to także istotne w przypadku algorytmów klasteryzacji, które posługują się odległością między punktami danych. W takim przypadku cecha, której zakres wartości to [-200;200] będzie wywierała znacznie większy wpływ na wynik grupowania niż cecha o zakresie wartości [-1;1]. 

Oczywiście niekiedy taka sytuacja moze być pożądana. Wówczas różne cechy można skalować w różny sposób np. [-1;1],[-5;5], itd., lub pozostawić bez skalowania.

Moduł scikit-learn oferuje tę funkcjonalność w postaci obiektu [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
?MinMaxScaler

#### Przykład dla pojedynczej cechy (kolumny danych)

Poniżej utworzona zostanie macierz zawierająca liczby zmienno przecinkowe.

In [3]:
x = np.array([0.1, 10., 15., -5.1, 6.2], ndmin=2).transpose()
x.shape

(5, 1)

In [4]:
x

array([[ 0.1],
       [10. ],
       [15. ],
       [-5.1],
       [ 6.2]])

Utworzenie obiektu klasy `MinMaxScaler` wykonuje się poprzez konstruktor klasy `MinMaxScaler`. 

In [18]:
my_scaler = MinMaxScaler(clip=True)

W chwili obecnej obiekt `my_scaler` jest "pusty" i nie zawiera żadnych danych. W celu jego pełnego określenia należy użyć metody `fit`, a jako argumentu użyć macierzy z wartościami.

In [19]:
my_scaler.fit(x)

MinMaxScaler(clip=True)

W pełni określony scaler posiada pola charakteryzujące dane wejściowe, np. wartość maksymalną i minimalną:

In [20]:
[my_scaler.data_min_, my_scaler.data_max_]

[array([-5.1]), array([15.])]

W celu dokonania transformacji danych zgodnie z zasadami określonymi przez `my_scaler` należy użyć metody `transform`. Poniżej zastosowanie tej metody do tablicy x, na podstawie, której utworzono scaler:

In [21]:
xs = my_scaler.transform(x)
xs

array([[0.25870647],
       [0.75124378],
       [1.        ],
       [0.        ],
       [0.56218905]])

Poniżej wykonano skalowanie dla nowych danych (np. zbiór testowy):

In [22]:
x_nowy = np.array([5., 2., -6, 16.],ndmin=2).transpose()
x_nowy

array([[ 5.],
       [ 2.],
       [-6.],
       [16.]])

Transformacja nowych danych.

**Uwaga:** W przypadku wprowadzania nowych danych, które zawierają wartości większe (mniejsze) od maksimum (minimum) zaobserwowanego w danych zwrócone wartości moga okazać się większe (mniejsze) od założonych granic, tu: [0;1]. Jest to poprawne. Jeżeli wartości mają być przycięte do danego zakresu to w konsruktorze należy przekazac argument `clip=True`: `my_scaler = MinMaxScaler(clip=True)` (od wersji sklearn 0.24)

In [17]:
x_nowy_s = my_scaler.transform(x_nowy)
x_nowy_s

array([[ 0.50248756],
       [ 0.35323383],
       [-0.04477612],
       [ 1.04975124]])

#### MinMaxScaler może działać na więcej niż jednej kolumnie, wówczas skaluje każdą cechę niezależnie

In [23]:
X = np.array([[ 2. , 6.  , 200.  , -450.,  105.  , 344.1, -33.],
              [ 0.1, 0.01,   0.02,    0.02,  0.01,   0.04,  0.032]]).transpose()
X

array([[ 2.000e+00,  1.000e-01],
       [ 6.000e+00,  1.000e-02],
       [ 2.000e+02,  2.000e-02],
       [-4.500e+02,  2.000e-02],
       [ 1.050e+02,  1.000e-02],
       [ 3.441e+02,  4.000e-02],
       [-3.300e+01,  3.200e-02]])

Utworzenie nowego scalera i wypisanie wartości minimalnych i maksymalnych

In [28]:
my_scaler1 = MinMaxScaler(clip=True)
my_scaler1.fit(X)
[my_scaler1.data_min_, my_scaler1.data_max_]

[array([-4.5e+02,  1.0e-02]), array([3.441e+02, 1.000e-01])]

In [29]:
Xs = my_scaler1.transform(X)
Xs

array([[0.56919783, 1.        ],
       [0.57423498, 0.        ],
       [0.81853671, 0.11111111],
       [0.        , 0.11111111],
       [0.69890442, 0.        ],
       [1.        , 0.33333333],
       [0.52512278, 0.24444444]])

Poniżej wykonano skalowanie dla nowych danych (np. zbiór testowy):

In [30]:
X_nowy =  np.array([[100.,   45., -102.],
                    [  0.001, 0.11,  0.125 ]]).transpose()
X_nowy

array([[ 1.00e+02,  1.00e-03],
       [ 4.50e+01,  1.10e-01],
       [-1.02e+02,  1.25e-01]])

In [31]:
X_nowy_s = my_scaler1.transform(X_nowy)
X_nowy_s

array([[0.69260798, 0.        ],
       [0.62334719, 1.        ],
       [0.43823196, 1.        ]])

### Skalowane kolumn do różnych zakresów

W części modeli możliwe jest wpynięcie na istotność cechy poprzez zmianę jej zakresu. Zazwyczaj cecha o większym zakresie będzie ważniejsza (np. będzie miała większy wpływ na obliczenie odległości między punktami). W takim przypadku możliwe jest przeskalowanie różnych cech do różnych zakresów wartości.

Poniżej utworzono dwa różne slcalery i przeskalowano każdą kolumnę oddzielnie, a następnie dokonano konkatenacji nowych kolumn.

In [32]:
X = np.array([[ 2. , 6.  , 200.  , -450.,  105.  , 344.1, -33.],
              [ 0.1, 0.01,   0.02,    0.02,  0.01,   0.04,  0.032]]).transpose()
X

array([[ 2.000e+00,  1.000e-01],
       [ 6.000e+00,  1.000e-02],
       [ 2.000e+02,  2.000e-02],
       [-4.500e+02,  2.000e-02],
       [ 1.050e+02,  1.000e-02],
       [ 3.441e+02,  4.000e-02],
       [-3.300e+01,  3.200e-02]])

In [33]:
my_scalerA = MinMaxScaler(feature_range=(-1,1))
my_scalerB = MinMaxScaler(feature_range=(-10,10))

In [34]:
X[:,0].reshape((-1,1))

array([[   2. ],
       [   6. ],
       [ 200. ],
       [-450. ],
       [ 105. ],
       [ 344.1],
       [ -33. ]])

In [35]:
my_scalerA.fit(X[:,0].reshape((-1,1)))

MinMaxScaler(feature_range=(-1, 1))

In [36]:
my_scalerB.fit(X[:,1].reshape((-1,1)))

MinMaxScaler(feature_range=(-10, 10))

In [37]:
Xs1 = np.concatenate((my_scalerA.transform(X[:,0].reshape((-1,1))), my_scalerB.transform(X[:,1].reshape((-1,1)))),axis=1)
Xs1

array([[  0.13839567,  10.        ],
       [  0.14846997, -10.        ],
       [  0.63707342,  -7.77777778],
       [ -1.        ,  -7.77777778],
       [  0.39780884, -10.        ],
       [  1.        ,  -3.33333333],
       [  0.05024556,  -5.11111111]])